In [103]:
import os
import rdflib
import networkx as nx
from pyvis.network import Network
import webbrowser

# Step 1: Define the style
style = {
    "color": "gray",
    "shape": "ellipse",
    "size": 10
}

# Step 2: Define the construct_subgraph function
def construct_subgraph(file_path, query, output_file):
    """
    Executes a CONSTRUCT query on an RDF graph and saves the resulting subgraph to a file.

    Args:
        file_path (str): Path to the input RDF Turtle file.
        query (str): SPARQL CONSTRUCT query string.
        output_file (str): Path to the output Turtle file.
    """
    # Load the original graph
    g = rdflib.Graph()
    g.parse(file_path, format='turtle')

    # Execute the CONSTRUCT query
    subgraph = g.query(query)

    # Get the resulting graph
    subgraph_graph = rdflib.Graph()
    for triple in subgraph:
        subgraph_graph.add(triple)

    # Save the subgraph to a Turtle file
    subgraph_graph.serialize(destination=output_file, format='turtle')

    print(f"Subgraph saved to {output_file}")

# Step 3: Convert the Graph to Results Format
def graph_to_results(graph):
    results = []
    for subj, pred, obj in graph:
        row = [('subject', subj), ('predicate', pred), ('object', obj)]
        results.append(row)
    return results

# Step 4: Helper Function to Extract Local Names
def get_local_name(uri):
    if isinstance(uri, rdflib.term.URIRef):
        uri = str(uri)
    if '#' in uri:
        return uri.split('#')[-1]
    elif '/' in uri:
        return uri.rstrip('/').split('/')[-1]
    else:
        return uri

# Step 5: Visualization Function
def visualize_query_results_interactive(results):
    G = nx.DiGraph()
    node_types = {}

    # Build the graph
    for row in results:
        row_dict = dict(row)
        subj = get_local_name(row_dict['subject'])
        pred = row_dict['predicate']
        obj = get_local_name(row_dict['object'])
        
        # Capture rdf:type relationships to identify node types
        if str(pred) == rdflib.RDF.type:
            node_types[subj] = obj

        G.add_edge(subj, obj, label=get_local_name(pred))

    # Initialize the Network object
    net = Network(
        notebook=True, height="1000px", width="100%",
        bgcolor="#ffffff", font_color="black", directed=True,
        cdn_resources='remote'
    )

    # Set visualization options and incorporate the style
    net.set_options(f"""
    var options = {{
        "nodes": {{
            "shape": "{style['shape']}",
            "color": "{style['color']}",
            "size": {style['size']},
            "font": {{
                "size": 14,
                "face": "Tahoma"
            }}
        }},
        "edges": {{
            "arrows": {{
                "to": {{
                    "enabled": true,
                    "scaleFactor": 1
                }}
            }},
            "smooth": {{
                "type": "continuous"
            }}
        }},
        "layout": {{
            "hierarchical": {{
                "enabled": true,
                "levelSeparation": 250,
                "nodeSpacing": 200,
                "treeSpacing": 300,
                "blockShifting": true,
                "edgeMinimization": true,
                "parentCentralization": true,
                "direction": "LR",
                "sortMethod": "hubsize"
            }}
        }},
        "physics": {{
            "enabled": false
        }}
    }}
    """)

    # Add nodes to the network
    for node in G.nodes():
        node_type_uri = node_types.get(node, None)
        node_type = get_local_name(node_type_uri) if node_type_uri else 'Unknown'
        net.add_node(node, label=node, title=f"Type: {node_type}", **style)

    # Add edges to the network
    for u, v, data in G.edges(data=True):
        label = data.get('label', '')
        net.add_edge(u, v, label=label, title=label)

    # Generate and show the network
    output_html = 'slegospace/ontologyspace/interactive_graph.html'
    net.show(output_html)
    webbrowser.open('file://' + os.path.realpath(output_html))
    print(f"Visualization saved to {output_html} and opened in your default browser.")

def extract_and_visualize_subgraph(file_path, subgraph_file, query):
    """
    Extracts and visualizes a subgraph based on a SPARQL CONSTRUCT query.

    Args:
        file_path (str): Path to the input RDF Turtle file.
        subgraph_file (str): Path to the output Turtle file where the subgraph will be saved.
        query (str): SPARQL CONSTRUCT query string to extract the subgraph.
    """
    # Construct and save the subgraph
    g = rdflib.Graph()
    g.parse(file_path, format='turtle')
    subgraph = g.query(query)
    subgraph_graph = rdflib.Graph()
    for triple in subgraph:
        subgraph_graph.add(triple)
    subgraph_graph.serialize(destination=subgraph_file, format='turtle')

    print(f"Subgraph saved to {subgraph_file}")

    # Parse the subgraph
    g = rdflib.Graph()
    g.parse(subgraph_file, format='turtle')

    print(f"Parsed {len(g)} triples from the subgraph.")

    # Convert the subgraph to results
    results = graph_to_results(g)

    # Visualize the subgraph
    visualize_query_results_interactive(results)

# Example usage of the function
file_path = 'slegospace/ontologyspace/hfd.ttl'
subgraph_file = 'slegospace/ontologyspace/hfd_subgraph.ttl'

query = """
PREFIX dbr: <http://example.org/class/>
PREFIX rdf1: <http://example.org/link/>

CONSTRUCT {
  ?subject1 ?predicate1 dbr:Liquidity .
  dbr:Liquidity ?predicate2 ?object1 .
  ?subject2 ?predicate3 dbr:DollarVolumeTraded .
  dbr:DollarVolumeTraded ?predicate4 ?object2 .
}
WHERE {
  {
    ?subject1 ?predicate1 dbr:Liquidity .
  }
  UNION
  {
    dbr:Liquidity ?predicate2 ?object1 .
  }
  UNION
  {
    ?subject2 ?predicate3 dbr:DollarVolumeTraded .
  }
  UNION
  {
    dbr:DollarVolumeTraded ?predicate4 ?object2 .
  }
}
"""

# Call the function with the query as a parameter
extract_and_visualize_subgraph(file_path, subgraph_file, query)



Subgraph saved to slegospace/ontologyspace/hfd_subgraph.ttl
Parsed 6 triples from the subgraph.
slegospace/ontologyspace/interactive_graph.html
Visualization saved to slegospace/ontologyspace/interactive_graph.html and opened in your default browser.


In [102]:
import os
import re
import rdflib
import networkx as nx
from pyvis.network import Network
import webbrowser

# Style definition remains the same
style = {
    "color": "gray",
    "shape": "ellipse",
    "size": 10
}

def get_local_name(uri):
    if uri is None:
        return "None"
    if isinstance(uri, rdflib.term.URIRef):
        uri = str(uri)
    if '#' in uri:
        return uri.split('#')[-1]
    elif '/' in uri:
        return uri.rstrip('/').split('/')[-1]
    else:
        return uri

def detect_query_type(query):
    # Remove comments and normalize whitespace
    query = re.sub(r'#.*$', '', query, flags=re.MULTILINE)
    query = ' '.join(query.split())
    
    # Check for CONSTRUCT
    if re.search(r'\bconstruct\b', query, re.IGNORECASE):
        return 'CONSTRUCT'
    # Check for SELECT
    elif re.search(r'\bselect\b', query, re.IGNORECASE):
        return 'SELECT'
    else:
        raise ValueError("Unable to determine query type. Please ensure the query is either CONSTRUCT or SELECT.")

def construct_subgraph(file_path, query):
    g = rdflib.Graph()
    g.parse(file_path, format='turtle')
    subgraph = g.query(query)
    subgraph_graph = rdflib.Graph()
    for triple in subgraph:
        subgraph_graph.add(triple)
    return subgraph_graph

def execute_select_query(file_path, query):
    g = rdflib.Graph()
    g.parse(file_path, format='turtle')
    results = g.query(query)
    return results

def graph_to_network(graph):
    G = nx.DiGraph()
    node_types = {}
    for subj, pred, obj in graph:
        subj_name = get_local_name(subj)
        obj_name = get_local_name(obj)
        pred_name = get_local_name(pred)
        
        if str(pred) == str(rdflib.RDF.type):
            node_types[subj_name] = obj_name

        G.add_edge(subj_name, obj_name, label=pred_name)
    
    return G, node_types

def select_results_to_network(select_results):
    G = nx.Graph()
    for row in select_results:
        for i, var in enumerate(select_results.vars):
            value = row[i] if i < len(row) else None
            node1 = f"{var}: {get_local_name(value)}"
            G.add_node(node1)
            for j, var2 in enumerate(select_results.vars):
                if i != j:
                    value2 = row[j] if j < len(row) else None
                    node2 = f"{var2}: {get_local_name(value2)}"
                    G.add_edge(node1, node2)
    return G

def visualize_network(G, node_types=None, is_directed=True):
    net = Network(notebook=True, height="1000px", width="100%", bgcolor="#ffffff", font_color="black", directed=is_directed)

    net.set_options(f"""
    var options = {{
        "nodes": {{
            "shape": "{style['shape']}",
            "color": "{style['color']}",
            "size": {style['size']},
            "font": {{
                "size": 14,
                "face": "Tahoma"
            }}
        }},
        "edges": {{
            "arrows": {{
                "to": {{
                    "enabled": {str(is_directed).lower()},
                    "scaleFactor": 1
                }}
            }},
            "smooth": {{
                "type": "continuous"
            }}
        }},
        "layout": {{
            "hierarchical": {{
                "enabled": true,
                "levelSeparation": 250,
                "nodeSpacing": 200,
                "treeSpacing": 300,
                "blockShifting": true,
                "edgeMinimization": true,
                "parentCentralization": true,
                "direction": "LR",
                "sortMethod": "hubsize"
            }}
        }},
        "physics": {{
            "enabled": false
        }}
    }}
    """)

    for node in G.nodes():
        node_type = node_types.get(node, 'Unknown') if node_types else 'N/A'
        net.add_node(node, label=node, title=f"Type: {node_type}", **style)

    for u, v, data in G.edges(data=True):
        label = data.get('label', '')
        net.add_edge(u, v, label=label, title=label)

    output_html = 'slegospace/ontologyspace/interactive_graph.html'
    net.show(output_html)
    webbrowser.open('file://' + os.path.realpath(output_html))
    print(f"Visualization saved to {output_html} and opened in your default browser.")

def extract_and_visualize(file_path, query):
    """
    Automatically detects query type, extracts results, and visualizes them based on a SPARQL query.

    Args:
        file_path (str): Path to the input RDF Turtle file.
        query (str): SPARQL query string (CONSTRUCT or SELECT).
    """
    query_type = detect_query_type(query)
    print(f"Detected query type: {query_type}")

    if query_type == 'CONSTRUCT':
        subgraph = construct_subgraph(file_path, query)
        G, node_types = graph_to_network(subgraph)
        visualize_network(G, node_types, is_directed=True)
    elif query_type == 'SELECT':
        select_results = execute_select_query(file_path, query)
        G = select_results_to_network(select_results)
        visualize_network(G, is_directed=True)
    else:
        raise ValueError("Unsupported query type detected.")

# Example usage
file_path = 'slegospace/ontologyspace/hfd.ttl'

# CONSTRUCT query example
construct_query = """
PREFIX dbr: <http://example.org/class/>
PREFIX rdf1: <http://example.org/link/>

CONSTRUCT {
  ?subject1 ?predicate1 dbr:Liquidity .
  dbr:Liquidity ?predicate2 ?object1 .
  ?subject2 ?predicate3 dbr:DollarVolumeTraded .
  dbr:DollarVolumeTraded ?predicate4 ?object2 .
}
WHERE {
  {
    ?subject1 ?predicate1 dbr:Liquidity .
  }
  UNION
  {
    dbr:Liquidity ?predicate2 ?object1 .
  }
  UNION
  {
    ?subject2 ?predicate3 dbr:DollarVolumeTraded .
  }
  UNION
  {
    dbr:DollarVolumeTraded ?predicate4 ?object2 .
  }
}
"""

# SELECT query example
select_query = """
PREFIX dbr: <http://example.org/class/>
PREFIX rdf1: <http://example.org/link/>
SELECT ?predicate ?object WHERE {
dbr:IntradayMetric ?predicate ?object .} 

"""

# Use either query
# extract_and_visualize(file_path, construct_query)
extract_and_visualize(file_path, select_query)

Detected query type: SELECT
slegospace/ontologyspace/interactive_graph.html
Visualization saved to slegospace/ontologyspace/interactive_graph.html and opened in your default browser.
